In [1]:
import PyPDF2

# Open the PDF file
with open("100 konkretow-pdf.pdf", "rb") as file:
    # Initialize PDF reader
    pdf_reader = PyPDF2.PdfReader(file)

    first_content_page = 0
    last_content_page = 32

    pages = []
    for page_num in range(first_content_page, last_content_page):
        text = pdf_reader.pages[page_num].extract_text()
        pages.append(text)



In [2]:
import re

In [3]:
print(pages[1])

2
www.100konkretow.plSpis treści:
Rodzina         3
M ł o d z i         4
Seniorzy         5
Kobiety         6
Przedsiębiorcy       7
Zdrowie         9
Edukacja         10
Polityka społeczna      12
P o d a t k i         1 4
Mieszkania        15
Energetyka        16
Unia Europejska       17
Środowisko        18
Świeckie Państwo      20
Rolnictwo         21
Rozliczenie rządów PiS     23
Praworządność       27
Bezpieczeństwo       28
Obrona narodowa      29
Organizacje pozarządowe    31
Kultura         32


In [4]:
categories_lines = "\n".join(pages[1].split("\n")[2:])
categories = categories_lines.split("\n")
categories[8]

'P o d a t k i         1 4'

In [10]:
import regex
categories_lines = "\n".join(pages[1].split("\n")[2:])
categories = categories_lines.split("\n")

def extract_category(s):
    match = regex.search(r"([\p{L}\s]*)\s+(\d\s?\d*$)", s)
    category, start_page = match.group(1).strip(), int(match.group(2).replace(" ", "").strip())
    if str.count(category, " ") > 2:
        category = category.replace(" ", "")
    return category, start_page 

categories = [extract_category(s) for s in categories]
categories =[
    (*categories[i], last_content_page + 1 if i == len(categories) - 1 else categories[i + 1][1])
    for i in range(len(categories))
]
categories

[('Rodzina', 3, 4),
 ('Młodzi', 4, 5),
 ('Seniorzy', 5, 6),
 ('Kobiety', 6, 7),
 ('Przedsiębiorcy', 7, 9),
 ('Zdrowie', 9, 10),
 ('Edukacja', 10, 12),
 ('Polityka społeczna', 12, 14),
 ('Podatki', 14, 15),
 ('Mieszkania', 15, 16),
 ('Energetyka', 16, 17),
 ('Unia Europejska', 17, 18),
 ('Środowisko', 18, 20),
 ('Świeckie Państwo', 20, 21),
 ('Rolnictwo', 21, 23),
 ('Rozliczenie rządów PiS', 23, 27),
 ('Praworządność', 27, 28),
 ('Bezpieczeństwo', 28, 29),
 ('Obrona narodowa', 29, 31),
 ('Organizacje pozarządowe', 31, 32),
 ('Kultura', 32, 33)]

In [11]:
results = {}
for category, start_page, end_page in categories:
    print(category)
    category_pages = pages[start_page - 1:end_page - 1]
    pattern = r'.*Nasze postulaty:\n(.+)$'
    all_statements = "\n".join(category_pages)
    match = re.search(pattern, all_statements, re.DOTALL)
    if match:
        text = match.group(1)
    else:
        text = all_statements
    category_statements = text.replace(u'\xa0', " ").split("\n")

            
    statement = ""
    category_merged = []
    for line in category_statements:
        statement += line
        if statement.endswith("."):
            replacements = ["dalsza część na kolejnej stronie", "(\d+)?(www.100konkretow.pl)(\d+)?", "Rozliczymyrządy PiS"]
            statement = re.sub(r"\s+", " ", statement)
            for replacement in replacements:
                statement = re.sub(replacement, "", statement)
            
            statement = regex.sub(r"\." + category, ".\n", statement)
            statement = regex.sub(category.strip() + r"( c\.d\.)", "", statement)
            statement = regex.sub(r" c\.d\.", "", statement)
            statements = [s.strip().replace("•", "\n•") for s in statement.split("\n")]
            
            category_merged.extend(statements)


            statement = ""
    category_merged = list(filter(
        lambda s: len(s) > 0,
        category_merged
    ))
    results[category] = category_merged
results

Rodzina
Młodzi
Seniorzy
Kobiety
Przedsiębiorcy
Zdrowie
Edukacja
Polityka społeczna
Podatki
Mieszkania
Energetyka
Unia Europejska
Środowisko
Świeckie Państwo
Rolnictwo
Rozliczenie rządów PiS
Praworządność
Bezpieczeństwo
Obrona narodowa
Organizacje pozarządowe
Kultura


{'Rodzina': ['Na stałe wprowadzimy finansowanie in vitro z budżetu państwa. Na finansowanie procedury in vitro przeznaczymy nie mniej niż 500 mln zł. – tak by pary, które chcą mieć dzieci miały dostęp do najnowocześniejszych metod.',
  'Wspieramy kobiety wracające na rynek pracy po urodzeniu dziecka: w ramach programu „Aktywna mama” wypłacimy 1500 zł miesięcznie na opiekę nad dzieckiem – tzw. „babciowe”.',
  'Natychmiast przystąpimy do realizacji polityki antyprzemocowej.',
  'Wprowadzimy obligacje antyinflacyjne poprzez zmianę w ustawie o NBP (gotowy projekt ustawy). Oszczędności Polaków będą zabezpieczone przed inflacją od pierwszego dnia.',
  'Wprowadzimy ustawę o związkach partnerskich.'],
 'Młodzi': ['Wprowadzimy 600 zł dopłaty na wynajem mieszkania dla młodych.',
  'Wprowadzimy kredyt z oprocentowaniem 0% na zakup pierwszego mieszkania.',
  'Przywrócimy finansowanie Telefonu Zaufania dla Dzieci i Młodzieży 116 111.',
  'Odwołamy Mikołaja Pawlaka z funkcji Rzecznika Praw Dziecka i

In [12]:
import json
with open("results.json", "w", encoding="utf-8") as file:
    json.dump(results, file, ensure_ascii=False, indent=4)

In [3]:
import json
with open("results.json", "rt", encoding="utf-8") as file:
    results = json.load(file)
    results = {
        category: [{"promise": statement, "fulfilled": False} for statement in statements]
        for category, statements in results.items()
    }
    with open("results_fulfilled.json", "wt", encoding="utf-8") as file:
        json.dump(results, file, ensure_ascii=False, indent=4)